In [33]:
import pandas as pd

file_path = 'mse_434_paper_data.xlsx'

try:
    df = pd.read_excel(file_path, sheet_name='customers')
    print("Data loaded successfully:")
    display(df)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully:


,Plant,Longitude,Latitude,Daily consumption [T],Safety stock [T]
0,Central,17.00,52.00,NaN,NaN
1,2,17.00,54.44,6.0,6.0
2,7,16.96,51.66,18.0,18.0
3,16,15.99,54.00,16.0,16.0
4,20,20.01,53.27,7.0,7.0
5,23,22.83,50.75,6.0,6.0


In [ ]:
'''
Calculate distance using the Haversine Formula for spherical data / longitude and latitude coordinates
'''

# https://community.esri.com/t5/coordinate-reference-systems-blog/distance-on-a-sphere-the-haversine-formula/ba-p/902128

def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    km = round(km, 3)
    
    return km


In [83]:
n = len(df)
# Convert Plant column to string to ensure consistent types
plant_names = df['Plant'].astype(str).tolist()
distance_matrix = pd.DataFrame(index=plant_names, columns=plant_names)
distance_matrix.index.name = None

# Create distance matrix using haversine() function to show distances between plants and customers (regional warehouses)
for i in range(n):
    for j in range(n):
        coord1 = (df.loc[i, 'Longitude'], df.loc[i, 'Latitude'])
        coord2 = (df.loc[j, 'Longitude'], df.loc[j, 'Latitude'])

        distance_matrix.iloc[i, j] = haversine(coord1, coord2)

# Convert all values to float
distance_matrix = distance_matrix.astype(float)

# Display result
print("Distance Matrix (in kilometers):")
display(distance_matrix)

Distance Matrix (in kilometers):


,Central,2,7,16,20,23
Central,0.000,271.316,37.906,232.427,247.357,427.723
2,271.316,0.000,309.133,81.885,236.396,568.338
7,37.906,309.133,0.000,268.226,273.348,421.152
16,232.427,81.885,268.226,0.000,277.147,587.949
20,247.357,236.396,273.348,277.147,0.000,340.192
23,427.723,568.338,421.152,587.949,340.192,0.000


In [127]:
# === Constants ===
T = 10  # number of time periods
L = 25  # vehicle capacity (tons)
M = 10000  # Big M, large enough to exceed any demand sum
b = 1.2  # inventory holding cost per ton
c_per_km = 2  # transportation cost per km

# === Load customers ===
customers_df = pd.read_excel("mse_434_paper_data.xlsx", sheet_name="customers")
customers = [str(i) for i in customers_df["Plant"] if str(i).lower() != "central"]

# === Load valid routes ===
routes_df = pd.read_excel("mse_434_paper_data.xlsx", sheet_name="valid_routes")
route_ids = routes_df["Route ID"].tolist()
route_map = {r: list(map(str, eval(custs))) for r, custs in zip(routes_df["Route ID"], routes_df["Route Combination"])}

# Cost Maps
cj = {}

for r in route_ids:
    route_customers = route_map[r]  
    
    # Calculate actual route distance: Central -> customer1 -> customer2 -> ... -> Central
    total_km = 0
    
    if len(route_customers) > 0:
        # Filter out customers not in distance matrix
        valid_customers = [str(i) for i in route_customers if str(i) in distance_matrix.columns]
        
        if valid_customers:
            # Distance from Central to first customer
            total_km += distance_matrix.loc['Central', valid_customers[0]]
            
            # Distance between consecutive customers
            for i in range(len(valid_customers) - 1):
                current_customer = valid_customers[i]
                next_customer = valid_customers[i + 1]
                total_km += distance_matrix.loc[current_customer, next_customer]
            
            # Distance from last customer back to Central
            total_km += distance_matrix.loc[valid_customers[-1], 'Central']
    
    cj[r] = round(c_per_km * total_km, 2) 
    

# Build aij: parameter for whether customer i is on route j
aij = {(i, j): int(i in route_map[j]) for i in customers for j in route_ids}

# # Build di (demand), gi (safety stock), bi (inventory cost per unit)
di = {i: customers_df.loc[customers_df["Plant"] == int(i), "Daily consumption [T]"].values[0] for i in customers}
gi = {i: customers_df.loc[customers_df["Plant"] == int(i), "Safety stock [T]"].values[0] for i in customers}
bi = {i: b for i in customers}


In [ ]:
from gurobipy import Model, GRB

model = Model("JointInventoryTransportation")

# === Indices ===
K = range(1, T + 1)

# === Variables ===

# Route usage decision 
x = model.addVars(route_ids, K, vtype=GRB.BINARY, name="x")                  # x_jk

# Inventory level at customer i in time period k
y = model.addVars(customers, K, lb=0, name="y")                              # y_ik

# Delivered amount at customer i in time period k on route j
z = model.addVars(customers, route_ids, K, lb=0, name="z")                   # z_ijk

# Objective: transport cost + inventory cost
model.setObjective(
    sum(cj[j] * x[j, k] for j in route_ids for k in K) +
    sum(bi[i] * y[i, k] for i in customers for k in K),
    GRB.MINIMIZE
)

In [ ]:
# # === Constraint (1): Initial inventory y_i1 = gi_i === (Initial inventory constraint)
# for i in customers:
#     model.addConstr(y[i, 1] == gi[i], name=f"init_inventory_{i}")

# === Constraint (2): z_ijk ≤ aij * x_jk * M === (Deliver only if the route is used)
for i in customers:
    for j in route_ids:
        for k in K:
            model.addConstr(z[i, j, k] <= aij[i, j] * x[j, k] * M, name=f"link_z_x_{i}_{j}_{k}")

# === Constraint (3): sum_i z_ijk ≤ Q for each route j, period k === (Truck capacity constraint)
for j in route_ids:
    for k in K:
        model.addConstr(sum(z[i, j, k] for i in customers) <= L, name=f"truck_cap_{j}_{k}")

# === Constraint (4): Inventory balance for period 1 to T-1 === (Inventory continuity constraint for periods 1 to T-1)
for i in customers:
    for k in range(1, T):
        model.addConstr(
            y[i, k+1] == y[i, k] + sum(z[i, j, k] for j in route_ids) - di[i],
            name=f"inventory_flow_{i}_{k}"
        )

# === Constraint (5): Cyclic inventory from T to 1 === (Cyclic inventory constraint for period T to 1)
for i in customers:
    model.addConstr(
        y[i, 1] == y[i, T] + sum(z[i, j, T] for j in route_ids) - di[i],
        name=f"cyclic_inventory_{i}"
    )

# === Constraint (6): Minimum inventory ≥ gi === (Safety stock constraint)
for i in customers:
    for k in K:
        model.addConstr(y[i, k] >= gi[i], name=f"safety_stock_{i}_{k}")
        
# F = 3  # number of trucks available per day

# # Limit number of routes (fleet size) per time period (EXTENSION)
# for k in K:
#     model.addConstr(sum(x[j, k] for j in route_ids) <= F, name=f"fleet_limit_{k}")

# Extension Idea: Constraint the number of trucks + decision variable for serving through common carrier if distance is too long
# Similar problem 



In [133]:
model.optimize()

# Output
print(f"\nTotal cost: {model.ObjVal:.2f}\n")

print("Route usage:")
for k in K:
    print(f"\nPeriod {k}:")
    for j in route_ids:
        if x[j, k].X > 0.5:
            print(f"  Route {j} used")

print("\nInventory levels:")
for i in customers:
    for k in K:
        print(f"Customer {i}, Period {k} → {y[i, k].X:.2f} units")

print("\nDeliveries z_ijk:")
for i in customers:
    for j in route_ids:
        for k in K:
            if z[i, j, k].X > 0.01:
                print(f"Deliver {z[i, j, k].X:.2f} to Customer {i} via Route {j} in Period {k}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 880 rows, 830 columns and 2320 nonzeros
Model fingerprint: 0x0516f230
Variable types: 700 continuous, 130 integer (130 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 2e+01]
Found heuristic solution: objective 66710.450000
Presolve removed 700 rows and 430 columns
Presolve time: 0.01s
Presolved: 180 rows, 400 columns, 670 nonzeros
Variable types: 270 continuous, 130 integer (130 binary)

Root relaxation: objective 1.715897e+04, 286 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 17158.9720    0   50 66710.4500 